In [ ]:
# Dependencies
import json
import requests
import pprint
from scipy.stats import linregress
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
#Import data as pandas DF's
#Education_2016_df = pd.read_csv("Resources/2016_Education_Postcode.csv")
#Income_2016_df = pd.read_csv("Resources/2016_Income_Postcode.csv")
#VehicleCount_2016_df = pd.read_csv("Resources/2016_VehicleCount_Postcode.csv")
Education_2021_df = pd.read_csv("Resources/2021_Education_Postcode.csv")
Income_2021_df = pd.read_csv("Resources/2021_Income_Postcode.csv")
VehicleCount_2021_df = pd.read_csv("Resources/2021_VehicleCount_Postcode.csv")

#adjust name once year confirmed
FuelType_2021_df = pd.read_csv("Resources/FuelType_Postcode.csv")
#pd.read_csv("Resources/2016_FuelType_Postcode.csv")

In [ ]:
def create_scatter(df, y_variable, x_variable):

    x_values = df[x_variable]
    y_values = df[y_variable]
    
    #get plot axis range for annotation location.
    ymin = df[y_variable].min()
    xmin = df[x_variable].min()

    ymax = df[y_variable].max()
    xmax = df[x_variable].max()

    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

    plt.scatter(
        x_values,
        y_values,
        color = 'blue',
        marker = 'o',
        s = 50,
        edgecolors = 'black')

    plt.plot(x_values, regress_values, "r-", linewidth = 1)

    #plt.annotate(line_eq, (xmin + 1, ymin + 5), fontsize =10, color = "red")

    plt.xlabel(x_variable)
    plt.ylabel(y_variable)
    plt.suptitle(f"The r-value is: {rvalue}", fontsize = 8, x = 0.2, y = 0.95)
    plt.annotate(line_eq, xy=(0.8, 1.05), fontsize =8, color = "red",
            xycoords='axes fraction', textcoords='axes fraction')

    plt.show()


In [ ]:
def reshape_df(df):
    return df.pivot_table(index = ['Vehicle_type', 'Postcode'], 
                          columns = 'Fuel_type', 
                          values = 'Count').reset_index().rename_axis(None, axis=1)

In [ ]:
def clean_headers(df):
    df.columns = df.columns.str.replace(" ", "_")
    return df

Clean Data

In [ ]:
#Clean Headers
#Education_2016_df = clean_headers(Education_2016_df)
#Income_2016_df = clean_headers(Income_2016_df)
#VehicleCount_2016_df = clean_headers(VehicleCount_2016_df)
Education_2021_df = clean_headers(Education_2021_df)
Income_2021_df = clean_headers(Income_2021_df)
VehicleCount_2021_df = clean_headers(VehicleCount_2021_df)
FuelType_2021_df = clean_headers(FuelType_2021_df)

In [ ]:
FuelType_2021_df = reshape_df(FuelType_2021_df)
#FuelType_2016_df = reshape_df(FuelType_2016_df)


In [ ]:
FuelType_2021_df = FuelType_2021_df[FuelType_2021_df["Vehicle_type"] != "Total"]
FuelType_2021_df = FuelType_2021_df[FuelType_2021_df["Postcode"] != "Total"]


Merge DFs so that Education, income, vehicle count, fuel type are in 1 df on postcode handle different postcodes that arent present in all datasets

In [ ]:
#Combine the 2016 data into a single dataset.
#data_education_income_2016 = pd.merge(Education_2016_df, Income_2016_df, how="inner", on= ["Postcode", "Postcode"])
#data_complete_2016 = pd.merge(data_education_income_2016, VehicleCount_2016_df ,how="inner", on= ["Postcode", "Postcode"])
#data_complete_2016 = data_complete_2016.drop(['State','Total_y', 'Total'], axis=1)

#data_complete_2016.head()

In [ ]:
Education_2021_df['Postcode'] = Education_2021_df['Postcode'].astype(int)
data_education_income_2021 = pd.merge(Education_2021_df, Income_2021_df, how="inner", on= ["Postcode", "Postcode"])
data_complete_2021 = pd.merge(data_education_income_2021, VehicleCount_2021_df ,how="inner", on= ["Postcode", "Postcode"])
data_complete_2021 = data_complete_2021.drop(['State','Total_y', 'Total'], axis=1)

Remove data for "Not Stated or Not Applicable" and subtract from Total column

In [ ]:
#Education_2016_df["Total"] = Education_2016_df["Total"] - (Education_2016_df["Not_stated"] + Education_2016_df["Not_applicable"])
#Education_2016_df = Education_2016_df.drop(columns=["Not_stated", "Not_applicable"])

Education_2021_df["Total"] = Education_2021_df["Total"] - (Education_2021_df["Not_stated"] + Education_2021_df["Not_applicable"])
Education_2021_df = Education_2021_df.drop(columns=["Not_stated", "Not_applicable"])


In [ ]:
#Income_2016_df["Total"] = Income_2016_df["Total"] - (Income_2016_df["Not_stated"] + Income_2016_df["Not_applicable"])
#Income_2016_df = Income_2016_df.drop(columns=["Not_stated", "Not_applicable"])

Income_2021_df["Total"] = Income_2021_df["Total"] - (Income_2021_df["Not_stated"] + Income_2021_df["Not_applicable"])
Income_2021_df = Income_2021_df.drop(columns=["Not_stated", "Not_applicable"])

In [ ]:
#Same for people with multiple cars?
Education_2021_df

In [ ]:
#Same for vehicle types, probably want to filter out
#FuelType_2021_df = FuelType_2021_df[FuelType_2021_df['Vehicle_type'] == 'Passenger Vehicles']

Create DF's with normalised values (percentage of total)

In [ ]:
Education_2021_Normalized = Education_2021_df.copy()
for col in Education_2021_Normalized.columns[2:-1]:
    Education_2021_Normalized[col] = Education_2021_Normalized[col] / Education_2021_Normalized["Total"] * 100
Education_2021_Normalized = Education_2021_Normalized.round(2)


In [ ]:
FuelType_2021_Normalized = FuelType_2021_df.copy()
for col in FuelType_2021_Normalized.columns[2:-1]:
    FuelType_2021_Normalized[col] = FuelType_2021_Normalized[col] / FuelType_2021_Normalized["Total"] * 100
FuelType_2021_Normalized = FuelType_2021_Normalized.round(2)

#if total = 0 then drop row?
FuelType_2021_Normalized

In [ ]:
Income_2021_Normalized = Income_2021_df.copy()
for col in Income_2021_Normalized.columns[2:-1]:
    Income_2021_Normalized[col] = Income_2021_Normalized[col] / Income_2021_Normalized["Total"] * 100
Income_2021_Normalized = Income_2021_Normalized.round(2)
Income_2021_Normalized

In [ ]:
#Calculate Education score for postcodes

Group into bigger bins for simplicity of plotting.
Make new dfs with created bins

In [ ]:
#Calculate an "Income score" for each 
def calculate_income_score(row):
    values = row[2:-2].values
    weights = [0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
    weighted_sum = sum([value * weight for value, weight in zip(values, weights)])
    
    return weighted_sum

Income_2021_Normalized['Income_Score'] = Income_2021_Normalized.apply(calculate_income_score, axis=1)
Income_2021_Normalized

In [ ]:
Education_2021_Scored = Education_2021_Normalized
#Apply a multiplier based on education score to new columns
Education_2021_Scored["Postgraduate_Degree_Score"] = Education_2021_Scored["Postgraduate_Degree_Level"].apply(lambda x: x*7)
Education_2021_Scored["Graduate_Diploma_and_Graduate_Certificate_Score"] = Education_2021_Scored["Graduate_Diploma_and_Graduate_Certificate_Level"].apply(lambda x: x*6)
Education_2021_Scored["Bachelor_Degree_Score"] = Education_2021_Scored["Bachelor_Degree_Level"].apply(lambda x: x*5)
Education_2021_Scored["Advanced_Diploma_and_Diploma_Score"] = Education_2021_Scored["Advanced_Diploma_and_Diploma_Level"].apply(lambda x: x*4)
Education_2021_Scored["Certificate_III_&_IV_Level_Score"] = Education_2021_Scored["Certificate_III_&_IV_Level"].apply(lambda x: x*3)
Education_2021_Scored["Certificate_I_&_II_Level_Score"] = Education_2021_Scored["Certificate_I_&_II_Level"].apply(lambda x: x*2)
Education_2021_Scored["Secondary_Education_-_Years_10_and_above_Score"] = Education_2021_Scored["Secondary_Education_-_Years_10_and_above"].apply(lambda x: x*2)
Education_2021_Scored["Secondary_Education_-_Years_9_and_below_Score"] = Education_2021_Scored["Secondary_Education_-_Years_9_and_below"].apply(lambda x: x*1)
#Drop unneeeded columns
Education_2021_Scored = Education_2021_Scored.drop(["Postgraduate_Degree_Level", "Graduate_Diploma_and_Graduate_Certificate_Level", "Bachelor_Degree_Level", "Advanced_Diploma_and_Diploma_Level", "Certificate_III_&_IV_Level", "Certificate_I_&_II_Level", "Secondary_Education_-_Years_10_and_above", "Secondary_Education_-_Years_9_and_below"], axis=1)
#Create a column in df with summed education scores
Education_Score_List = ["Postgraduate_Degree_Score", "Graduate_Diploma_and_Graduate_Certificate_Score", "Bachelor_Degree_Score", "Advanced_Diploma_and_Diploma_Score", "Certificate_III_&_IV_Level_Score", "Certificate_I_&_II_Level_Score", "Secondary_Education_-_Years_10_and_above_Score", "Secondary_Education_-_Years_9_and_below_Score"]
Education_2021_Scored["Education_Score"] = Education_2021_Scored[Education_Score_List].sum(axis=1)


Merge Education and Income tables with fuel types, Then remove suburbs where no fuel types were recorded to disregard NaN values.

In [ ]:
Income_Fueltype_2021 = pd.merge(FuelType_2021_Normalized, Income_2021_Normalized, on='Postcode')
#Merge Regional vs Metro here------------------- GJP
Income_Fueltype_2021 = Income_Fueltype_2021[Income_Fueltype_2021['Total_x'] != 0]
Income_Fueltype_2021

In [ ]:
Education_Fueltype_2021 = pd.merge(FuelType_2021_Normalized, Education_2021_Scored, on = "Postcode")
#GJP Merge regional and metro table here too, then filter can filter for metro etc. and adjust scatter plots accordingly
Education_Fueltype_2021 = Education_Fueltype_2021[Education_Fueltype_2021['Total_x'] != 0]
Education_Fueltype_2021


In [ ]:
# read in and add electoraterating values for each postcode. Remove duplicates and keep the first non-null value for electoraterating
# Outer Metro, Inner Metro and provincial are grouped for this purpose as "Metro", and Rural is left as Rural. Null values will be dropped for the purpose of plotting.

Metro_Rural_Postcode = pd.read_csv('Resources/ElectorateRating_Postcode.csv')
Metro_Rural_Postcode = Metro_Rural_Postcode.drop_duplicates().groupby('Postcode', as_index=False).first()
Metro_Rural_Postcode

In [ ]:
#Merge Metro/Rural values with Income and Education dfs
Education_Fueltype_2021 = pd.merge(Education_Fueltype_2021, Metro_Rural_Postcode, on = "Postcode")
Income_Fueltype_2021 = pd.merge(Income_Fueltype_2021, Metro_Rural_Postcode, on='Postcode')
Education_Fueltype_2021

In [ ]:
#Drop Rows where Metro_Rural is NaN
Education_Fueltype_2021.dropna(subset=["Metro_Rural"], inplace=True)
Income_Fueltype_2021.dropna(subset=["Metro_Rural"], inplace=True)

In [ ]:
Income_Fueltype_2021.sort_values("Electric", ascending = False)

Create Plots

All values Scatter

In [ ]:
create_scatter(Income_Fueltype_2021, "Electric", "Income_Score")

In [ ]:
create_scatter(Education_Fueltype_2021, "Electric", "Education_Score")

Scatter filtered for vehicles in metro areas

In [ ]:
Income_Metro = Income_Fueltype_2021[Income_Fueltype_2021['Metro_Rural'] == 'Metro']
Education_Metro = Education_Fueltype_2021[Education_Fueltype_2021['Metro_Rural'] == 'Metro']

In [ ]:
create_scatter(Education_Metro.query("Total_x > 1000"), "Electric", "Education_Score")
create_scatter(Education_Metro.query("Total_x > 5000"), "Electric", "Education_Score")

In [ ]:
create_scatter(Income_Metro.query("Total_x > 1000"), "Electric", "Income_Score")
create_scatter(Income_Metro.query("Total_x > 5000"), "Electric", "Income_Score")